In [ ]:
# Install a pip package in the current Jupyter kernel to format data into dataframes
import sys
!{sys.executable} -m pip install pandas

# Install a pip package to perform the html document parsing and extract information
!{sys.executable} -m pip install bs4

# Install a pip package to export the data in an excel format
!{sys.executable} -m pip install openpyxl

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, time, datetime, re, random

In [7]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_best-selling_Nintendo_Switch_video_games"
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
response = requests.get(wiki_url, headers=headers)
response.status_code

200

In [12]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
soup

# Find the table of switch sales 
html_table =soup.find('table',{'class':"wikitable"})
html_table

# Convert the wikipedia table into a pandas dataframe
switch_sales_df = pd.read_html(str(html_table))[0]
switch_sales_df

,Rank,Title,Copies sold,As of,Release date[a],Genre(s),Developer(s),Publisher(s)
0,1,Mario Kart 8 Deluxe,53.79 million[4],"March 31, 2023","April 28, 2017",Kart racing,Nintendo EPD,Nintendo
1,2,Animal Crossing: New Horizons,42.21 million[4],"March 31, 2023","March 20, 2020",Social simulation,Nintendo EPD,Nintendo
2,3,Super Smash Bros. Ultimate,31.09 million[4],"March 31, 2023","December 7, 2018",Fighting,Bandai Namco StudiosSora Ltd.,Nintendo
3,4,The Legend of Zelda: Breath of the Wild,29.81 million[4],"March 31, 2023","March 3, 2017",Action-adventure,Nintendo EPD,Nintendo
4,5,Pokémon Sword and Shield,25.82 million[4],"March 31, 2023","November 15, 2019",Role-playing,Game Freak,The Pokémon CompanyNintendo
...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,1 million[25],"September 8, 2020","December 20, 2018",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
74,72,Fitness Boxing 2: Rhythm and Exercise,1 million[26],"December 9, 2021","December 4, 2020",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
75,72,Shin Megami Tensei V,1 million[27],"April 18, 2022","November 11, 2021",Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo
76,72,Story of Seasons: Pioneers of Olive Town,1 million[28],"November 18, 2021","February 25, 2021",Simulationrole-playing,Marvelous,Xseed Games


In [23]:
def copies_sold_to_numeric(value):
    parsed_string = re.sub(r'million(\[[a-z]*[0-9]*\])*', '', value)
    return float(parsed_string) * 1000000

def convert_date(date):
        #           Month day, Year
    format_type = '%B %d, %Y'  # The format it currently is
    datetime_str = datetime.datetime.strptime(date, format_type)
    return datetime_str.strftime("%m/%d/%Y") # mm/dd/yyyy

def create_metacritic_url(title):
    mod_title = re.sub(r'[.:,\']', "", title)
    mod_title = mod_title.replace(" ", "-")
    mod_title = mod_title.replace("é", "e")
    return 'https://www.metacritic.com/game/switch/' + mod_title.lower() + "/details"

In [26]:
# Mutate variables to better fit data standards
switch_sales_df_formatted = switch_sales_df.assign(
    Release_date = lambda df: df['Release date[a]'].map(lambda var:  convert_date(var) ),
    As_of = lambda df: df['As of'].map(lambda var:  convert_date(var) ),
    Copies_sold = lambda df: df['Copies sold'].map(lambda var:  copies_sold_to_numeric(var)),
    Metacritic_url = lambda df: df['Title'].map(lambda var:  create_metacritic_url(var) )
).drop(columns=['Release date[a]', 'As of', 'Copies sold'], axis= 1)
switch_sales_df_formatted

,Rank,Title,Genre(s),Developer(s),Publisher(s),Release_date,As_of,Copies_sold,Metacritic_url
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,04/28/2017,03/31/2023,53790000.0,https://www.metacritic.com/game/switch/mario-k...
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/20/2020,03/31/2023,42210000.0,https://www.metacritic.com/game/switch/animal-...
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,12/07/2018,03/31/2023,31090000.0,https://www.metacritic.com/game/switch/super-s...
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/03/2017,03/31/2023,29810000.0,https://www.metacritic.com/game/switch/the-leg...
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,11/15/2019,03/31/2023,25820000.0,https://www.metacritic.com/game/switch/pokemon...
...,...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/20/2018,09/08/2020,1000000.0,https://www.metacritic.com/game/switch/fitness...
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/04/2020,12/09/2021,1000000.0,https://www.metacritic.com/game/switch/fitness...
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,11/11/2021,04/18/2022,1000000.0,https://www.metacritic.com/game/switch/shin-me...
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,02/25/2021,11/18/2021,1000000.0,https://www.metacritic.com/game/switch/story-o...


In [41]:
req = requests.get("https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details", headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

description = soup.find("div", class_ = "summary_detail product_summary").find("span", class_='data').text
meta_score = soup.find("div", class_ = "metascore_wrap feature_metascore").find("span").text
meta_ratings = soup.find("div", class_ = "metascore_wrap feature_metascore").find("div",class_="summary").find("p").find("span", class_="count").find("a").find("span").text
user_score = soup.find("div", class_ = "userscore_wrap feature_userscore").find("a", class_ = "metascore_anchor").find("div").text
user_ratings = soup.find("div", class_ = "userscore_wrap feature_userscore").find("div",class_="summary").find("span", class_="count").text

meta_ratings = re.findall('[0-9]+', meta_ratings)[0]
user_ratings = re.findall('[0-9]+', user_ratings)[0]

{'description' : description, 
            'meta_score' :  float(meta_score) , 
            'meta_ratings' : int(meta_ratings), 
            'user_score' : float(user_score), 
            'user_ratings' : int(user_ratings) }


{'description': "Race and battle your friends in the definitive version of Mario Kart 8. Hit the road with the definitive version of Mario Kart 8 and play anytime, anywhere! Race your friends or battle them in a revised battle mode on new and returning battle courses. Play locally in up to 4-player multiplayer in 1080p while playing in TV Mode. Every track from the Wii U version, including DLC, makes a glorious return. Plus, the Inklings appear as all-new guest characters, along with returning favorites, such as King Boo, Dry Bones, and Bowser Jr. Race your friends in the definitive version of Mario Kart 8, only on Nintendo Switch. Race as every character on every track from the Wii U version, including DLC characters and tracks. Pop some balloons in the revamped Battle mode, complete with Balloon Battle and Bob-omb Blast. Battle on new courses, like Urchin Underpass and Battle Stadium, or returning ones, such as GCN Luigi's Mansion and SNES Battle Course 1. Inkling Girl & Inkling Boy 

In [45]:
def scrape_metacritic(html):
    soup = BeautifulSoup(html, 'html.parser')

    description = soup.find("div", class_ = "summary_detail product_summary").find("span", class_='data').text
    meta_score = soup.find("div", class_ = "metascore_wrap feature_metascore").find("span").text
    meta_ratings = soup.find("div", class_ = "metascore_wrap feature_metascore").find("div",class_="summary").find("p").find("span", class_="count").find("a").find("span").text
    user_score = soup.find("div", class_ = "userscore_wrap feature_userscore").find("a", class_ = "metascore_anchor").find("div").text
    user_ratings = soup.find("div", class_ = "userscore_wrap feature_userscore").find("div",class_="summary").find("span", class_="count").text

    meta_ratings = re.findall('[0-9]+', meta_ratings)[0]
    user_ratings = re.findall('[0-9]+', user_ratings)[0]

    return {'description' : description, 
                'meta_score' :  float(meta_score) , 
                'meta_ratings' : int(meta_ratings), 
                'user_score' : float(user_score), 
                'user_ratings' : int(user_ratings) }

def perform_random_delay(delay = 3, random_offset = .5):
    time.sleep(delay + random.uniform(0,random_offset))

In [46]:
urls = switch_sales_df_formatted["Metacritic_url"].tolist()
metacritic_results  = []

for url in urls:
    # Go to the URL
    req = requests.get(url, headers=headers)
    # If the url is successfully reached, then attempt to scrape the page
    if req.status_code == 200:
        # PERFORM SCRAPING LOGIC HERE
        try:
            curr_url_result = scrape_metacritic(req.text)
            curr_url_result['Metacritic_url'], curr_url_result['status_code'] = url, 200
            curr_url_result['parsed'] = True
        except: 
            # If the url is not valid, then return an empty dicitonary and denote the status code
            curr_url_result = {'description' : '','meta_score' : '',  'meta_ratings' : '', 
                           'user_score' : '', 'user_ratings' :'', 'status_code' :  200,
                           'Metacritic_url' : url, 'parsed' : False}
    else: 
        # If the url is not valid, then return an empty dicitonary and denote the status code
        curr_url_result = {'description' : '','meta_score' : '',  'meta_ratings' : '', 
                           'user_score' : '', 'user_ratings' :'', 'status_code' :  req.status_code,
                           'Metacritic_url' : url, 'parsed' : False}
    # append the result
    metacritic_results.append(curr_url_result)
    # perform a random delay between scrapes
    perform_random_delay()

In [48]:
reviews_df  = pd.DataFrame(metacritic_results)
reviews_df

,description,meta_score,meta_ratings,user_score,user_ratings,Metacritic_url,status_code,parsed
0,Race and battle your friends in the definitive...,92.0,95,8.6,2698,https://www.metacritic.com/game/switch/mario-k...,200,True
1,If the hustle and bustle of modern life’s got ...,90.0,111,5.6,6583,https://www.metacritic.com/game/switch/animal-...,200,True
2,"Inklings from the Splatoon series, as well as ...",93.0,99,8.6,3780,https://www.metacritic.com/game/switch/super-s...,200,True
3,Forget everything you know about The Legend of...,97.0,109,8.7,20388,https://www.metacritic.com/game/switch/the-leg...,200,True
4,,,,,,https://www.metacritic.com/game/switch/pokemon...,404,False
...,...,...,...,...,...,...,...,...
73,"Get off the couch and get moving with fun, box...",66.0,14,6.5,50,https://www.metacritic.com/game/switch/fitness...,200,True
74,,,,,,https://www.metacritic.com/game/switch/fitness...,404,False
75,The ambitions of god and human clash amidst th...,84.0,98,7.5,749,https://www.metacritic.com/game/switch/shin-me...,200,True
76,Inspired by tales of their grandfather's pione...,71.0,60,7.1,88,https://www.metacritic.com/game/switch/story-o...,200,True


In [49]:
# Merge the two dataframes together
switch_sales_reviews_df  = switch_sales_df_formatted.merge(reviews_df)
switch_sales_reviews_df

,Rank,Title,Genre(s),Developer(s),Publisher(s),Release_date,As_of,Copies_sold,Metacritic_url,description,meta_score,meta_ratings,user_score,user_ratings,status_code,parsed
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,04/28/2017,03/31/2023,53790000.0,https://www.metacritic.com/game/switch/mario-k...,Race and battle your friends in the definitive...,92.0,95,8.6,2698,200,True
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/20/2020,03/31/2023,42210000.0,https://www.metacritic.com/game/switch/animal-...,If the hustle and bustle of modern life’s got ...,90.0,111,5.6,6583,200,True
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,12/07/2018,03/31/2023,31090000.0,https://www.metacritic.com/game/switch/super-s...,"Inklings from the Splatoon series, as well as ...",93.0,99,8.6,3780,200,True
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/03/2017,03/31/2023,29810000.0,https://www.metacritic.com/game/switch/the-leg...,Forget everything you know about The Legend of...,97.0,109,8.7,20388,200,True
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,11/15/2019,03/31/2023,25820000.0,https://www.metacritic.com/game/switch/pokemon...,,,,,,404,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/20/2018,09/08/2020,1000000.0,https://www.metacritic.com/game/switch/fitness...,"Get off the couch and get moving with fun, box...",66.0,14,6.5,50,200,True
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/04/2020,12/09/2021,1000000.0,https://www.metacritic.com/game/switch/fitness...,,,,,,404,False
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,11/11/2021,04/18/2022,1000000.0,https://www.metacritic.com/game/switch/shin-me...,The ambitions of god and human clash amidst th...,84.0,98,7.5,749,200,True
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,02/25/2021,11/18/2021,1000000.0,https://www.metacritic.com/game/switch/story-o...,Inspired by tales of their grandfather's pione...,71.0,60,7.1,88,200,True


In [51]:
sum(switch_sales_reviews_df['parsed']) / switch_sales_reviews_df.shape[0]

0.8846153846153846

In [ ]:
switch_sales_reviews_df.to_excel("Switch Sales and Reviews.xlsx", index = False)
switch_sales_reviews_df.to_csv("Switch Sales and Reviews.csv", index = False)